---
title: "NIST (part 1): Parsing the spectral library"

date: last-modified

author:

- name: Ralf Gabriels

  orcid: 0000-0002-1679-1711

  affiliations:
    - VIB-UGent Center for Medical Biotechnology, VIB, Belgium
    - Department of Biomolecular Medicine, Ghent University, Belgium

bibliography: ../../references.bib

---

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ProteomicsML/ProteomicsML/blob/main/tutorials/fragmentation/_nist-1-parsing-spectral-library.ipynb)


In bottom-up proteomics, a peptide fragmentation spectrum (MS2) is the most
central source of information to identify a peptide (and the original protein by
extend). In traditional peptide fragment spectrum identification workflows,
only the presence and location (x-axis) of peaks in the spectrum are used to 
identify the peptide that generated the spectrum. The intensity of these peaks
(y-axis) are, however, seldomly used in a comprehensive manner. At most,
traditional approaches naively assume that higher intensity is always better.
The lack of usage of peptide spectrum intensity patterns in the identification
step is mainly due to their complexity. The location of certain peaks (e.g., b-
and y-ions) are easily calculated based on their mass for any given peptide.
Their intensity, however, follows complex yet predictable patterns that cannot
be simply calculated. Nevertheless, they can be predicted with machine learning.

[@gabriels2019]

In this tutorial you will learn the basic steps in developing a machine learning
predictor for peptide fragmentation intensity prediction. The first chapter
handles the preparation and parsing of training data and the second chapter
first handles training a traditional machine learning model, and then a deep
learning model.

To avoid an overly complex tutorial, some aspects to intensity prediction are
simplified or not handled. For example, the resulting models will only be able
to predict singly charged b- and y-ions for unmodified peptides.

In [ ]:
# Installing some required python packages
# ! pip install rich numpy pandas pyarrow matplotlib seaborn spectrum_utils==0.3.5 pyteomics sklearn hyperopt --quiet

## 1.1 Finding spectral libraries

Training data for peptide fragmentation spectrum intensity prediction consists
of spectra that were already identified. The most convenient source of such
information are spectral libraries. These are datasets that were compiled from
a collection of mass spectrometry runs and usually consist of a single
representative spectrum for each peptide that was identified.

Many precompiled spectral libraries are available online. You can also generate
your own from a collection of proteomics experiments, using software such as
SpectraST.

Spectral libraries can be downloaded, for instance, from NIST, the
[US National Institute of Standards and Technology](https://chemdata.nist.gov/dokuwiki/doku.php?id=peptidew:cdownload)
. For this part of the practical, we will download the
[2020 Human HCD library of "best" tryptic spectra](https://chemdata.nist.gov/dokuwiki/doku.php?id=peptidew:lib:humanhcd20160503). For ease-of-use, we will download it in the text-based NIST MSP format.

The following code cell automatically downloads and extracts the spectral
library file using Linux commands. If you are working locally on a Windows
machine, you can download and extract the files manually.

_Note the exclamation marks in the code cell below, which allow us to run Linux commands, instead of Python._

In [ ]:
# THIS CELL WORKS ONLY ON LINUX HOSTS (e.g. Google Colab)

# `wget` is a Linux command line tool to download files. 
! wget -q https://chemdata.nist.gov/download/peptide_library/libraries/human/HCD/2020_05_19/human_hcd_tryp_best.msp.tar.gz

# Next, we need to unpack the .tar.gz file
! tar -xf human_hcd_tryp_best.msp.tar.gz

Let's explore the MSP spectral library file, using the Linux `head` command:

In [ ]:
! head human_hcd_tryp_best.msp

This shows the beginning of the first spectrum in the spectral library. Each spectrum entry consists of a header with identification data and metadata, and a peak list with three columns:

 - m/z values
 - intensity values
 - peak annotation info


 As the sequence of the first peptide is `AAAAAAAAAAAAAAAGAGAGAK`, we can assume that this library is ordered alphabetically. You can read through the file to verify this assumption. This ordering will be important later on.

## 1.2 Parsing the MSP spectral library file

[Pyteomics](https://pyteomics.readthedocs.io/) is a Python package for proteomics that contains readers for many proteomics-related file formats. Unfortunately, MSP is not one of them. So first, we need a custom MSP reader function.

In [ ]:
from rich import print, progress  # Rich is a pretty cool library. Google it ;)
import numpy as np
import pandas as pd

This function iterates over each line in the MSP file. Once it has gathered all information for a single spectrum, it uses `yield` to return a dictionary. This means that we can iterate over the function using a `for` loop, and process spectra one-by-one. 

_If you do not fully understand the function, no problem! This is not the important part of the tutorial_ 😉

In [ ]:
def read_msp(filename):
    """Iterate over MSP spectral library file and return spectra as dicts."""
    spectrum = {}
    mz = []
    intensity = []
    annotation = []

    for line in open(filename, "rt"):
        # `Name: ` is the first line of a new entry in the file
        if line.startswith("Name: "):
            if spectrum:
                # Finalize and yield previous spectrum
                spectrum["sequence"] = spectrum["Fullname"].split(".")[1]  # Remove the previous/next amino acids
                spectrum["mz"] = np.array(mz, dtype="float32")
                spectrum["intensity"] = np.array(intensity, dtype="float32")
                spectrum["annotation"] = np.array(annotation, dtype="str")
                yield spectrum

                # Define new spectrum
                spectrum = {}
                mz = []
                intensity = []
                annotation = []
            
            # Extract everything after `Name: `
            spectrum["Name"] = line.strip()[6:]

        elif line.startswith("Comment: "):
            # Parse all comment items as metadata
            metadata = [i.split("=") for i in line[9:].split(" ")]
            for item in metadata:
                if len(item) == 2:
                    spectrum[item[0]] = item[1]

        elif line.startswith("Num peaks: "):
            spectrum["Num peaks"] = int(line.strip()[11:])

        elif len(line.split("\t")) == 3:
            # Parse peak list items one-by-one
            line = line.strip().split("\t")
            mz.append(line[0])
            intensity.append(line[1])
            annotation.append(line[2].strip('"'))

    # Final spectrum
    spectrum["sequence"] = spectrum["Fullname"].split(".")[1]  # Remove the previous/next amino acids
    spectrum["mz"] = np.array(mz, dtype="float32")
    spectrum["intensity"] = np.array(intensity, dtype="float32")
    spectrum["annotation"] = np.array(annotation, dtype="str")
    yield spectrum

Let's explore the first spectrum:

In [ ]:
# break allows us to only stop after the first spectrum is defined
for spectrum in read_msp("human_hcd_tryp_best.msp"):
    print(spectrum["Name"])
    break


We can format the peak list as a Pandas DataFrame:

In [ ]:
pd.DataFrame({
    "mz": spectrum["mz"],
    "intensity": spectrum["intensity"],
    "annotation": spectrum["annotation"],
})

The left-most column denotes the peak annotation. This tells us which ion generated the peak, according to the search engine or library generation software. Note that many peaks — highlighted with a question mark — are not annotated, even though the spectrum was confidently identified.

Using the Python package [spectrum_utils](https://spectrum-utils.readthedocs.io/), we can easily visualize the spectrum:


In [ ]:
import matplotlib.pyplot as plt

import spectrum_utils.spectrum as sus
import spectrum_utils.plot as sup

In [ ]:
plt.figure(figsize=(12,6))
sup.spectrum(
    sus.MsmsSpectrum(
        identifier=spectrum["Name"],
        precursor_mz=float(spectrum["Parent"]),
        precursor_charge=int(spectrum["Charge"]),
        mz=spectrum["mz"],
        intensity=spectrum["intensity"]
    )
)
plt.show()

## 1.3 Preparing spectra for training

To use a peptide fragmentation spectrum such as this one as training _target_ for a machine learning model, it needs some preparation and parsing. Usually this comprises of the following steps:

1. Normalize the intensities
2. Transform the intensities
3. Annotate the peaks
4. Parse the relevant peak intensities to an format suitable for machine learning

For each of these steps, we will write a function that can be reused later on in
the tutorial.

### 1.3.1 Normalize the intensities

Depending on the file format, peak intensities can range from 0 to 1, from 0 to 100, from 0 from 10 000... Machine learning algorithms require the target (and feature) values to be normalized in a specific range. For fragmentation spectra, there are two common options: total ion current (TIC) normalization and base peak normalization. For the former, all intensity values are divided by the total sum of all intensity values in the spectrum. The sum of all normalized intensities will be `1`. For the latter, all intensity values are divided by the most intense peak in the spectrum, resulting in that peak to have normalized intensity  `1`. Here we will implement TIC-normalization.

In [ ]:
def tic_normalize(msp_spectrum):
    tic = np.sum(msp_spectrum["intensity"])
    msp_spectrum["intensity"] = msp_spectrum["intensity"] / tic
    return msp_spectrum

In [ ]:
# Before normalization
spectrum["intensity"][:10]

In [ ]:
spectrum = tic_normalize(spectrum)

# After normalization
spectrum["intensity"][:10]

### 1.3.2 Transform the intensities

The distribution of peak intensities shows us that most peptide fragmentation peaks have a relatively low intensity, while only a few peaks are more intense:

In [ ]:
import seaborn as sns
sns.set_style("whitegrid")

# Before transform
sns.displot(spectrum["intensity"], bins=20)
plt.show()

To make the intensities follow a more linear distribution — which is better for machine learning algorithms — we can transform the intensity values. Two methods are often used:
square root-tranform, and log-transform. While both methods mostly have the same effect, we will here opt for square root transform, as log-transform results in negative values, which can be cumbersome to deal with.

In [ ]:
def sqrt_transform(msp_spectrum):
    msp_spectrum["intensity"] = np.sqrt(msp_spectrum["intensity"])
    return msp_spectrum

In [ ]:
spectrum = sqrt_transform(spectrum)

# After transform
sns.displot(spectrum["intensity"], bins=20)
plt.show()

### 1.3.3 Annotate the peaks

With the NIST spectral libraries, this step is pretty easy, as peak annotations are already present. If this would not be the case, we can make use of 
spectrum_utils, which can annotate peaks given the peptide _sequence_ and any _modifications_. See the [spectrum_utils documentation](https://spectrum-utils.readthedocs.io/en/latest/processing.html#peak-annotations) for more info.

In [ ]:
plt.figure(figsize=(12,6))
sup.spectrum(
    sus.MsmsSpectrum(
        identifier=spectrum["Name"],
        precursor_mz=float(spectrum["Parent"]),
        precursor_charge=int(spectrum["Charge"]),
        mz=spectrum["mz"],
        intensity=spectrum["intensity"],
        peptide=spectrum["sequence"],
    ).annotate_peptide_fragments(25, "ppm")
)
plt.show()

### 1.3.4 Parse the relevant peak intensities to an format suitable for machine learning

Note in the visualization above that spectrum_utils only annotated b- and y-ions, while in the MSP file many other ion types are also annotated. For simplicity's sake, in this tutorial we will train a model to only predict singly charged b- and y-ions. 

Let's filter the spectrum for only those peaks. This can be done with regular
expressions (regex) and numpy. TIP: [regex101.com](https://regex101.com)  is a great website for building and testing regular expressions. The regex
`^(b|y)([0-9]+)\/` only matches peak annotations for singly charged b- and
y-ions. You can investigate it at https://regex101.com/r/bgZ7EG/1.

In the following function, `numpy.vectorize` is used. What does it do and why do we use it here?

In [ ]:
import re

def filter_peaks(msp_spectrum):
    """Filter spectrum peaks to only charge 1 b- and y ions."""
    # Generate the boolean mask
    get_mask = np.vectorize(lambda x: bool(re.match("^(b|y)([0-9]+)\/", x)))
    mask = get_mask(msp_spectrum["annotation"])
    
    # Apply the mask to each peak array
    msp_spectrum["annotation"] = msp_spectrum["annotation"][mask]
    msp_spectrum["mz"] = msp_spectrum["mz"][mask]
    msp_spectrum["intensity"] = msp_spectrum["intensity"][mask]

    return msp_spectrum

spectrum = filter_peaks(spectrum)

In [ ]:
plt.figure(figsize=(12,6))
sup.spectrum(
    sus.MsmsSpectrum(
        identifier=spectrum["Name"],
        precursor_mz=float(spectrum["Parent"]),
        precursor_charge=int(spectrum["Charge"]),
        mz=spectrum["mz"],
        intensity=spectrum["intensity"],
        peptide=spectrum["sequence"]
    ).annotate_peptide_fragments(25, "ppm")
)
plt.show()

Now, the spectrum indeed only contains singly charged b- and y-ions. Note the nice gausian-like distributions of equally-distanced b- and y-ions. This is a feature specific for this peptide spectrum. Can you guess why? Tip: Take a look at the peptide sequence.

Currently, all peaks are listed together in single numpy arrays, sorted by m/z values. For training a machine learning model, we need the intensity values in a more suitable structure. As we are planning to only predict simple singly charged b- and y-ions, we can create two arrays — one for each ion type — with the ions sorted by ion number:

```python
parsed_intensity = {
    "b": [b1, b2, b3, b4 ... bN],
    "y": [y1, y2, y3, y4 ... yN]
}
```


where N is the total number of possible fragments for that peptide sequence. Quick question: What value will N have for our peptide with sequence `AAAAAAAAAAAAAAAGAGAGAK`?

The following function builds upon the `filter_peaks` function to not only filter the correct ion types, but also order them properly:


In [ ]:
def parse_peaks(msp_spectrum, ion_type):
    # Generate vectorized functions
    get_ions = np.vectorize(lambda x: bool(re.match(f"^({ion_type})([0-9]+)\/", x)))
    get_ion_order = np.vectorize(lambda x: re.match(f"^({ion_type})([0-9]+)\/", x)[2])

    # Get mask with requested ion types
    mask = get_ions(msp_spectrum["annotation"])

    # Create empty array with for all possible ions
    n_ions = len(msp_spectrum["sequence"]) - 1
    parsed_intensity = np.zeros(n_ions)

    # Check if any ions of this type are present
    if mask.any():
        # Filter for ion type and sort
        ion_order = get_ion_order(msp_spectrum["annotation"][mask]).astype(int) - 1
        # Add ions to correct positions in new array
        parsed_intensity[ion_order] = msp_spectrum["intensity"][mask]

    try:
        msp_spectrum["parsed_intensity"][ion_type] = parsed_intensity
    except KeyError:
        msp_spectrum["parsed_intensity"] = {}
        msp_spectrum["parsed_intensity"][ion_type] = parsed_intensity
    
    return msp_spectrum

spectrum = parse_peaks(spectrum, "b")
spectrum = parse_peaks(spectrum, "y")

In [ ]:
spectrum['parsed_intensity']

Great! These values are now ready to be used as prediction targets for a machine learning algorithm.

## 1.4 Parse the full spectral library

Now that all functions for spectrum preparation are written, we can parse the full spectral library. Let's first explore some of the basic statistics of this
library.

### 1.4.1 Exploring basic spectral library statistics

#### Total number of spectra

This one is easy, we quickly scan the number of lines starting with `Name: ` in the file:

In [ ]:
def msp_count_spectra(filename):
    """Count number of spectra in MSP file."""
    count = 0
    for line in open(filename, "rt"):
        if line.startswith("Name: "):
            count += 1
    return count

n_spectra = msp_count_spectra("human_hcd_tryp_best.msp")
print(n_spectra)

For more statistics, we will have to read the full library. One advantage we have now, is that knowing the total number of spectra allows us to track the progress of reading the full MSP file (`progress.track`). To limit the amount of data we keep in memory (this full MSP file is almost 2GB!), we can process the intensity values of each spectrum while parsing and only keep the parsed data:

In [ ]:
spectrum_list = []
for msp_spectrum in progress.track(
    read_msp("human_hcd_tryp_best.msp"),
    total=n_spectra,
    description="Parsing  MSP file..."
):
    # Process intensities
    msp_spectrum = tic_normalize(msp_spectrum)
    msp_spectrum = sqrt_transform(msp_spectrum)
    msp_spectrum = parse_peaks(msp_spectrum, "b")  # Adds `parsed_intensity` > `b`
    msp_spectrum = parse_peaks(msp_spectrum, "y")  # Adds `parsed_intensity` > `y`

    # Parse metadata
    spectrum = {
        "sequence": msp_spectrum["sequence"],
        "modifications": msp_spectrum["Mods"],
        "charge": int(msp_spectrum["Charge"]),
        "nce": float(msp_spectrum["NCE"]),
        "parsed_intensity": msp_spectrum["parsed_intensity"]
    }

    # Append to list
    spectrum_list.append(spectrum)

Generating a Pandas DataFrame from the list of spectrum dictionaries, allows us to easily explore the full dataset:

In [ ]:
spectrum_df = pd.DataFrame(spectrum_list)
spectrum_df

Making a Pandas DataFrame out of `spectrum_list` is so simple because it is a 
list of consistent dictionaries. 

#### Precursor charge state

A different precursor charge state can heavily alter peptide fragmentation. It is therefore important to have a representative amount of peptide spectra for each charge state in the spectral library.

In [ ]:
sns.countplot(data=spectrum_df, x="charge")
plt.show()

#### Peptide length

Idem for the length of the peptide sequence. It usually makes sense to filter 
the train dataset for peptides within a certain length range.

In [ ]:
sns.kdeplot(spectrum_df["sequence"].str.len())
plt.xlabel("Sequence length")
plt.show()

In [ ]:
spectrum_df["sequence"].str.len().describe()

In [ ]:
(spectrum_df["sequence"].str.len() > 35).value_counts()

For this dataset, the minimum peptide length is 6, while the maximum is 50.
Nevertheless, only 

##### Peptide modifications

Likewise, peptide modifications can influence peptide fragmentation. How many of the spectra in our library come from modified peptides?

In [ ]:
modification_state = (spectrum_df["modifications"] == "0").map({True: "Unmodified", False: "Modified"})
sns.countplot(x=modification_state)
plt.show()

#### Collision energy

In [ ]:
sns.histplot(spectrum_df["nce"], bins=30)
plt.xlabel("NCE")
plt.show()

Note the range of the x-axis, which was automatically chosen by the plotting library. It seems to start at 0, which indicates that some values are very low...

In [ ]:
(spectrum_df["nce"] == 0.0).value_counts()

Indeed, it seems that some peptide spectra have collision energy (CE) `0`, which most likely means that the true collision energy setting is not known. We can either opt to not use CE as a feature for training, or to remove these spectra from the dataset. Including these values would introduce unwanted noise in the training data.

#### Duplicate entries?

An important aspect to compiling training data for machine learning is whether or not entries are duplicated. With spectral libraries, matters are complicated by multiple levels of "uniqueness":

- Peptide level: Unique sequence
- Peptidoform level: Unique sequence & modifications
- Precursor level: Unique sequence & modifications & charge

More parameters can be included for "uniqueness", such as instrument and acquisition properties: CE, fragmentation method (beam-type CID ("HCD"), trap-type CID, ETD, EAD...), acquisition method (Orbitrap, ion trap, TOF...). In this tutorial, we are using only HCD Orbitrap data, which makes things a bit simpler. Nevertheless, this will impact the application domain of the final models.

In [ ]:
counts = pd.DataFrame({
    "Level": [
        "Full library",
        "Precursor",
        "Peptidoform",
        "Peptide",
    ],
    "Count": [
        spectrum_df.shape[0],
        spectrum_df[["sequence", "modifications", "charge"]].drop_duplicates().shape[0],
        spectrum_df[["sequence", "modifications"]].drop_duplicates().shape[0],
        spectrum_df["sequence"].unique().shape[0],
    ],
})

In [ ]:
sns.barplot(data=counts, x="Level", y="Count")
plt.show()

In [ ]:
counts

Seems like this library was filtered for uniqueness on the precursor level.

### 1.4.1 Selecting data

For selecting training data, we will apply some additional filters:

- While plain amino acid sequences are straightforward to encode, peptide modifications complicate matters. For simplicity's sake, we will therefore not open the "can of modifications" in this tutorial.
- As we might want to use CE as a feature, we can remove the small amount of entries that are missing the a CE value
- To make the training task a bit less complex, we can limit peptide length to 35. Although the maximum peptide length in this library is 50, only 4944 spectra have a peptide length of over 35.

In [ ]:
spectrum_df = spectrum_df[
    (modification_state == "Unmodified") &
    (spectrum_df["sequence"].str.len() <= 35) &
    (spectrum_df["nce"] != 0)
]

Let's see how many spectra we retained:

In [ ]:
spectrum_df.shape[0]

In [ ]:
spectrum

### 1.4.2 Train / validation / test split

Now that we have our data, we can filter it to a final set for training and validation and a final set for testing. A small reminder of what these terms mean:

- Training data: For training the model
- Validation data: For validating the model while optimizing hyperparameters
- Testing data: For final testing of model that was trained with the best hyperparameters (according to the validation data), right before deployment

The testing data cannot be used until a final model is trained, and serves as a last test before deployment. It should not be used before a final model is selected. 

In [ ]:
from sklearn.model_selection import train_test_split

np.random.seed()  # Why is this needed?

train_val_peptides, test_peptides = train_test_split(spectrum_df["sequence"].unique(), train_size=0.9)
train_val_spectra = spectrum_df[spectrum_df["sequence"].isin(train_val_peptides)]
test_spectra = spectrum_df[spectrum_df["sequence"].isin(test_peptides)]

Question: Why do we not apply `train_test_split()` directly on `spectrum_df`, but instead on `spectrum_df["sequence"].unique()`?